# Final Project

## Bayesian Classifier and Feature Importance

**Note:** This is a copy of Lab 2, so some features from that may still remain. I've done my best to clean it up to make it presentable though.

In [1]:
from pathlib import Path
home = str(Path.home()) # all other paths are relative to this path. change to something else if this is not the case on your system

In [2]:
#### NO NEED TO EDIT ####
from pathlib import Path
home = str(Path.home()) # all other paths are relative to this path. change to something else if this is not the case on your system
REPO = f"{home}/csc-466-student"
NOTEBOOK = "CSGO_Bayes"

%load_ext autoreload
%autoreload 2

from importlib import import_module
helper = import_module(f'{NOTEBOOK}_helper')

#### NO NEED TO EDIT ####

For this lab, we are going to first implement an empirical naive bayesian classifier, then implement feature importance measures and apply it to a dataset, and finally, we will examine the affect of modifying the priors.

For developing this lab, we can use famous Titanic Kaggle dataset. Description of the data is found https://www.kaggle.com/c/titanic/data.

In [3]:
import pandas as pd
import numpy as np
cs = pd.read_csv(
    f"csgo_games.csv"
)
cs

,match_date,team_1,team_2,t1_points,t2_points,t1_world_rank,t2_world_rank,t1_h2h_win_perc,t2_h2h_win_perc,winner,...,t2_player5_dpr,t2_player5_spr,t2_player5_opk_ratio,t2_player5_opk_rating,t2_player5_wins_perc_after_fk,t2_player5_fk_perc_in_wins,t2_player5_multikill_perc,t2_player5_rating_at_least_one_perc,t2_player5_is_sniper,t2_player5_clutch_win_perc
0,2016-12-18,CLOUD9,HELLRAISERS,13,16,9,20,0.500000,0.500000,t2,...,0.63,0.10,1.05,0.92,0.733,0.104,0.147018,0.528,False,0.666667
1,2016-12-18,IMMORTALS,G2,17,19,13,10,0.500000,0.500000,t2,...,0.69,0.09,0.85,0.96,0.739,0.147,0.135810,0.438,True,0.406250
2,2016-12-18,MOUSESPORTS,IMMORTALS,16,3,12,13,0.500000,0.500000,t1,...,0.67,0.07,0.79,0.94,0.723,0.129,0.148257,0.526,False,0.733333
3,2016-12-18,DIGNITAS,G2,16,9,6,10,0.416667,0.583333,t1,...,0.69,0.09,0.85,0.96,0.739,0.147,0.135810,0.438,True,0.406250
4,2016-12-18,OPTIC,HELLRAISERS,16,10,4,20,0.500000,0.500000,t1,...,0.63,0.10,1.05,0.92,0.733,0.104,0.147018,0.528,False,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3782,2020-01-25,VIRTUS.PRO,CLOUD9,0,2,19,17,0.500000,0.500000,t2,...,0.67,0.11,0.80,0.95,0.681,0.121,0.130481,0.444,False,0.777778
3783,2020-01-24,HEROIC,MAD LIONS,10,16,16,18,0.333333,0.666667,t2,...,0.65,0.08,0.64,0.80,0.811,0.082,0.095975,0.250,False,0.625000
3784,2020-01-19,HEROIC,FORZE,0,2,16,13,0.857143,0.142857,t2,...,0.63,0.09,0.89,0.92,0.747,0.113,0.154618,0.635,False,0.709677
3785,2020-01-19,CR4ZY,FORZE,0,2,20,13,0.538462,0.461538,t2,...,0.63,0.09,0.89,0.92,0.747,0.113,0.154618,0.635,False,0.709677


Here are some demonstrations on how we can group the data:

In [4]:
allStats = ['rating','impact','kdr','dmr','kpr','apr','dpr','spr','opk_ratio','opk_rating','wins_perc_after_fk',
            'fk_perc_in_wins','multikill_perc','rating_at_least_one_perc','is_sniper','clutch_win_perc']
print('All unique stats available')
print(allStats)

allPlayers = helper.get_players()
print('A list of all players on both teams')
print(allPlayers)

p2All = helper.get_player_stats(1, 2)
print('All stats for player 2 of team 1')
print(p2All)

print(cs.winner.unique())

All unique stats available
['rating', 'impact', 'kdr', 'dmr', 'kpr', 'apr', 'dpr', 'spr', 'opk_ratio', 'opk_rating', 'wins_perc_after_fk', 'fk_perc_in_wins', 'multikill_perc', 'rating_at_least_one_perc', 'is_sniper', 'clutch_win_perc']
A list of all players on both teams
['t1_player1', 't1_player2', 't1_player3', 't1_player4', 't1_player5', 't2_player1', 't2_player2', 't2_player3', 't2_player4', 't2_player5']
All stats for player 2 of team 1
['t1_player2_rating', 't1_player2_impact', 't1_player2_kdr', 't1_player2_dmr', 't1_player2_kpr', 't1_player2_apr', 't1_player2_dpr', 't1_player2_spr', 't1_player2_opk_ratio', 't1_player2_opk_rating', 't1_player2_wins_perc_after_fk', 't1_player2_fk_perc_in_wins', 't1_player2_multikill_perc', 't1_player2_rating_at_least_one_perc', 't1_player2_is_sniper', 't1_player2_clutch_win_perc']
['t2' 't1' 'draw']


Here we are doing some preprocessing by converting a boolean condition to an int and converting a string-based boolean to an int

In [5]:
for player in allPlayers:
    cs[player + "_is_sniper"] = cs[player + "_is_sniper"].astype(int)
    
cs.winner = cs.winner.map(dict(t1=1, draw=0, t2=-1))
cs = cs[cs.winner != 0]
cs

,match_date,team_1,team_2,t1_points,t2_points,t1_world_rank,t2_world_rank,t1_h2h_win_perc,t2_h2h_win_perc,winner,...,t2_player5_dpr,t2_player5_spr,t2_player5_opk_ratio,t2_player5_opk_rating,t2_player5_wins_perc_after_fk,t2_player5_fk_perc_in_wins,t2_player5_multikill_perc,t2_player5_rating_at_least_one_perc,t2_player5_is_sniper,t2_player5_clutch_win_perc
0,2016-12-18,CLOUD9,HELLRAISERS,13,16,9,20,0.500000,0.500000,-1,...,0.63,0.10,1.05,0.92,0.733,0.104,0.147018,0.528,0,0.666667
1,2016-12-18,IMMORTALS,G2,17,19,13,10,0.500000,0.500000,-1,...,0.69,0.09,0.85,0.96,0.739,0.147,0.135810,0.438,1,0.406250
2,2016-12-18,MOUSESPORTS,IMMORTALS,16,3,12,13,0.500000,0.500000,1,...,0.67,0.07,0.79,0.94,0.723,0.129,0.148257,0.526,0,0.733333
3,2016-12-18,DIGNITAS,G2,16,9,6,10,0.416667,0.583333,1,...,0.69,0.09,0.85,0.96,0.739,0.147,0.135810,0.438,1,0.406250
4,2016-12-18,OPTIC,HELLRAISERS,16,10,4,20,0.500000,0.500000,1,...,0.63,0.10,1.05,0.92,0.733,0.104,0.147018,0.528,0,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3782,2020-01-25,VIRTUS.PRO,CLOUD9,0,2,19,17,0.500000,0.500000,-1,...,0.67,0.11,0.80,0.95,0.681,0.121,0.130481,0.444,0,0.777778
3783,2020-01-24,HEROIC,MAD LIONS,10,16,16,18,0.333333,0.666667,-1,...,0.65,0.08,0.64,0.80,0.811,0.082,0.095975,0.250,0,0.625000
3784,2020-01-19,HEROIC,FORZE,0,2,16,13,0.857143,0.142857,-1,...,0.63,0.09,0.89,0.92,0.747,0.113,0.154618,0.635,0,0.709677
3785,2020-01-19,CR4ZY,FORZE,0,2,20,13,0.538462,0.461538,-1,...,0.63,0.09,0.89,0.92,0.747,0.113,0.154618,0.635,0,0.709677


Here are some more examples on how we can group the data:

In [6]:
p1Stats = ['t1_player1_' + stat for stat in allStats]
print('All stats for player 1 on team 1')
print(p1Stats)

allRating = []
for player in allPlayers:
    allRating.append(player + '_rating')
print('Rating for each player on both teams')
print(allRating)

allPlayersPerStat = helper.get_players_per_stat()
print('List of all players\' stats organized by each stat')
print(allPlayersPerStat)

All stats for player 1 on team 1
['t1_player1_rating', 't1_player1_impact', 't1_player1_kdr', 't1_player1_dmr', 't1_player1_kpr', 't1_player1_apr', 't1_player1_dpr', 't1_player1_spr', 't1_player1_opk_ratio', 't1_player1_opk_rating', 't1_player1_wins_perc_after_fk', 't1_player1_fk_perc_in_wins', 't1_player1_multikill_perc', 't1_player1_rating_at_least_one_perc', 't1_player1_is_sniper', 't1_player1_clutch_win_perc']
Rating for each player on both teams
['t1_player1_rating', 't1_player2_rating', 't1_player3_rating', 't1_player4_rating', 't1_player5_rating', 't2_player1_rating', 't2_player2_rating', 't2_player3_rating', 't2_player4_rating', 't2_player5_rating']
List of all players' stats organized by each stat
{'rating': ['t1_player1_rating', 't1_player2_rating', 't1_player3_rating', 't1_player4_rating', 't1_player5_rating', 't2_player1_rating', 't2_player2_rating', 't2_player3_rating', 't2_player4_rating', 't2_player5_rating'], 'impact': ['t1_player1_impact', 't1_player2_impact', 't1_play

An example dataframe when we group by one of the aforementioned groupings:

In [7]:
rating = cs.loc[:,allRating]
rating

,t1_player1_rating,t1_player2_rating,t1_player3_rating,t1_player4_rating,t1_player5_rating,t2_player1_rating,t2_player2_rating,t2_player3_rating,t2_player4_rating,t2_player5_rating
0,1.24,1.15,1.09,1.08,1.05,1.20,1.08,1.06,1.03,1.02
1,1.21,1.20,1.17,1.07,1.03,1.15,1.11,1.01,0.96,0.94
2,1.18,1.12,1.09,0.97,0.95,1.21,1.20,1.17,1.07,1.03
3,1.19,1.18,1.13,1.02,0.95,1.15,1.11,1.01,0.96,0.94
4,1.19,1.14,1.12,1.11,1.09,1.20,1.08,1.06,1.03,1.02
...,...,...,...,...,...,...,...,...,...,...
3782,1.20,1.13,1.11,0.94,0.93,1.17,1.16,1.15,1.15,0.95
3783,1.17,1.15,1.09,1.08,1.06,1.21,1.18,1.15,1.09,0.82
3784,1.15,1.12,1.10,1.06,1.05,1.20,1.15,1.10,1.05,1.03
3785,1.17,1.06,1.05,0.92,0.91,1.20,1.15,1.10,1.05,1.03


In [8]:
allPlayersPerStat = helper.get_players_per_stat()
teams = helper.get_teams()
print(teams)

(['t1_player1', 't1_player2', 't1_player3', 't1_player4', 't1_player5'], ['t2_player1', 't2_player2', 't2_player3', 't2_player4', 't2_player5'])


In [12]:
print('After Normilization')
cs_norm = helper.normalize_cs(cs)
cs_norm

After Normilization


,winner,world_rank,h2h_win_perc,rating,impact,kdr,dmr,kpr,apr,dpr,spr,opk_ratio,opk_rating,wins_perc_after_fk,fk_perc_in_wins,multikill_perc,rating_at_least_one_perc,is_sniper,clutch_win_perc
0,-1,-11,0.000000,0.22,0.22,0.28,5.4,0.09,0.06,-0.09,-0.01,0.61,0.03,0.130,-0.015,0.053051,0.368,0,0.270753
1,-1,3,0.000000,0.51,0.56,0.76,24.5,0.21,0.14,-0.26,-0.08,1.13,0.31,0.364,0.074,0.034550,0.748,-1,0.746143
2,1,-1,0.000000,-0.37,-0.33,-0.58,-21.9,-0.20,-0.11,0.14,0.04,-0.74,-0.15,-0.207,-0.021,-0.026775,-0.482,0,-0.760780
3,1,-4,-0.166667,0.30,0.32,0.45,10.1,0.09,0.07,-0.18,-0.06,0.47,0.12,0.271,0.009,0.016960,0.542,-1,1.061758
4,1,-16,0.000000,0.26,0.26,0.26,9.0,0.10,0.11,-0.06,0.05,0.19,0.07,-0.051,-0.017,0.047179,0.402,1,-0.177137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3782,-1,2,0.000000,-0.27,-0.44,-0.16,-17.4,-0.22,-0.07,-0.09,-0.22,0.48,0.05,-0.237,0.018,-0.074193,-0.321,0,-0.551432
3783,-1,-2,-0.333333,0.10,0.08,0.13,5.3,0.03,0.05,-0.04,-0.03,0.95,0.23,-0.085,0.049,-0.018188,0.054,0,-0.568354
3784,-1,3,0.714286,-0.05,-0.03,-0.12,-4.6,-0.09,0.08,-0.02,0.05,0.67,0.19,-0.038,0.026,-0.050476,-0.173,0,-0.050508
3785,-1,7,0.076923,-0.42,-0.44,-0.63,-24.4,-0.22,-0.05,0.17,0.03,-0.30,-0.07,-0.312,-0.014,-0.089147,-0.981,1,-0.011111


#### Testing various data
Here is our environment where I tested calculations for how likely it is for a team to win versus another. In order to test, I would replace the first argument of np.reshape with the matchup that was in question.

In [13]:
#fictional team vs real teams
dtvsH = [0, 0, 0.6,1.11,0.78,46.2,0.44,0,-0.05,-0.12,2.46,0.94,0.003,0.287,-0.008557123135,0.755,1,-0.6591318674]
dtvsTL = [0, 0, 0.82,1.25,1.04,54.7,0.55,0.04,-0.11,-0.02,1.95,0.86,-0.066,0.217,0.03962059328,1.127,1,-0.0659343924]
dtvsC9 = [0,0,0.58,1.1,0.65,51.8,0.52,0.04,0.08,-0.0,2.11,0.91,-0.417,0.233,-0.008133629319,0.954,1,-0.7156309377]
#real teams vs real teams
TLvsC9 = [0, 0, -0.24,-0.15,-0.39,-2.9,-0.03,0,0.19,0.01,0.16,0.05,-0.351,0.016,-0.0477542226,-0.173,0,-0.6496965453]
HvsC9 = [0,0,-0.02,-0.01,-0.13,5.6,0.08,0.04,0.13,0.11,-0.35,-0.03,-0.42,-0.054,0.000423493816,0.199,0,-0.05649907032]
HvsTL = [0,0,0.22,0.14,0.26,8.5,0.11,0.04,-0.06,0.1,-0.51,-0.08,-0.069,-0.07,0.04817771641,0.372,0,0.593197475]
#TL -oSee +s1mple roster change vs real teams
HvsnewTL = [0,0,0,-0.2,0,-9.4,-0.06,0.04,-0.06,0.1,-0.64,-0.29,-0.073,-0.148,-0.006796868748,0.006,0,0.2035870854]
C9vsnewTL = [0,0,0.02,-0.19,0.13,-15,-0.14,0,-0.19,-0.01,-0.29,-0.26,0.347,-0.094,-0.007220362564,-0.193,0,0.2600861557]
xColumns = ['world_rank', 'h2h_win_perc', 'rating', 'impact', 'kdr', 'dmr', 'kpr',
       'apr', 'dpr', 'spr', 'opk_ratio', 'opk_rating', 'wins_perc_after_fk',
       'fk_perc_in_wins', 'multikill_perc', 'rating_at_least_one_perc',
       'is_sniper', 'clutch_win_perc']
x = pd.DataFrame(data = np.reshape(C9vsnewTL, (1, 18)), index = [0], columns = xColumns)
x

,world_rank,h2h_win_perc,rating,impact,kdr,dmr,kpr,apr,dpr,spr,opk_ratio,opk_rating,wins_perc_after_fk,fk_perc_in_wins,multikill_perc,rating_at_least_one_perc,is_sniper,clutch_win_perc
0,0.0,0.0,0.02,-0.19,0.13,-15.0,-0.14,0.0,-0.19,-0.01,-0.29,-0.26,0.347,-0.094,-0.00722,-0.193,0.0,0.260086


In [14]:
norm_with_dummy = pd.concat([cs_norm.iloc[:0], x, cs_norm.iloc[0:]]).reset_index(drop=True)
norm_with_dummy
bin_cols = norm_with_dummy.columns
bin_cols = bin_cols.drop(['winner', 'is_sniper'])
cs_binned_d = helper.bin_cols(norm_with_dummy, bin_cols)
dummyNode = cs_binned_d.iloc[0]
dummyNode = dummyNode.drop('winner')
cs_binned_d = cs_binned_d.drop(index=0)
dummyNode

world_rank                        (-0.5, 6.0]
h2h_win_perc                    (-0.333, 0.0]
rating                           (0.0, 0.417]
impact                        (-0.61, -0.105]
kdr                               (0.0, 0.61]
dmr                           (-34.133, -9.9]
kpr                            (-0.17, 0.015]
apr                             (-0.07, 0.01]
dpr                             (-0.223, 0.0]
spr                           (-0.05, 0.0833]
opk_ratio                    (-1.053, -0.075]
opk_rating                   (-0.327, -0.025]
wins_perc_after_fk              (0.23, 0.537]
fk_perc_in_wins             (-0.164, -0.0903]
multikill_perc              (-0.0605, 0.0146]
rating_at_least_one_perc        (-0.607, 0.0]
is_sniper                                 0.0
clutch_win_perc               (0.0496, 0.665]
Name: 0, dtype: object

In [15]:
X = cs_binned_d.drop(["winner"],axis=1)
y = cs_binned_d["winner"]
probs = helper.class_conditional(X,y)
priors = helper.compute_priors(y)
post_probs = helper.posteriors(probs,priors,dummyNode)
post_probs

{'winner=-1.0|world_rank=(-0.5, 6.0],h2h_win_perc=(-0.333, 0.0],rating=(0.0, 0.417],impact=(-0.61, -0.105],kdr=(0.0, 0.61],dmr=(-34.133, -9.9],kpr=(-0.17, 0.015],apr=(-0.07, 0.01],dpr=(-0.223, 0.0],spr=(-0.05, 0.0833],opk_ratio=(-1.053, -0.075],opk_rating=(-0.327, -0.025],wins_perc_after_fk=(0.23, 0.537],fk_perc_in_wins=(-0.164, -0.0903],multikill_perc=(-0.0605, 0.0146],rating_at_least_one_perc=(-0.607, 0.0],is_sniper=0.0,clutch_win_perc=(0.0496, 0.665]': 0.8300475524701062,
 'winner=1.0|world_rank=(-0.5, 6.0],h2h_win_perc=(-0.333, 0.0],rating=(0.0, 0.417],impact=(-0.61, -0.105],kdr=(0.0, 0.61],dmr=(-34.133, -9.9],kpr=(-0.17, 0.015],apr=(-0.07, 0.01],dpr=(-0.223, 0.0],spr=(-0.05, 0.0833],opk_ratio=(-1.053, -0.075],opk_rating=(-0.327, -0.025],wins_perc_after_fk=(0.23, 0.537],fk_perc_in_wins=(-0.164, -0.0903],multikill_perc=(-0.0605, 0.0146],rating_at_least_one_perc=(-0.607, 0.0],is_sniper=0.0,clutch_win_perc=(0.0496, 0.665]': 0.16995244752989383}

For reference: Dream team has 4 of the highest performing players over 
the last year plus a world-class in-game leader (who has mediocre statistics)

Heroic is currently the #1 rated team in the world

Team Liquid is currently the #5 rated team in the world

Cloud9 is currently the #8 rated team in the world

Dream team vs Cloud9: 99.95% for Dream Team

Dream team vs Team Liquid: 99.997% for Dream Team

Dream team vs Heroic: 100% for Heroic

Heroic vs Team Liquid: 74.38% for Heroic 

Heroic vs Cloud9: 82.63% for Cloud9

Team Liquid vs Cloud9: 92% for Cloud9

New Team Liquid vs Heroic: 85.27% for new Team Liquid

New Team Liquid vs Cloud9: 83% for new Team Liquid

In [16]:
bin_cols = cs_norm.columns
bin_cols = bin_cols.drop(['winner', 'is_sniper'])
cs_binned = helper.bin_cols(cs_norm, bin_cols)
cs_binned

,winner,world_rank,h2h_win_perc,rating,impact,kdr,dmr,kpr,apr,dpr,spr,opk_ratio,opk_rating,wins_perc_after_fk,fk_perc_in_wins,multikill_perc,rating_at_least_one_perc,is_sniper,clutch_win_perc
0,-1,"(-13.5, -7.0]","(-0.333, 0.0]","(0.0, 0.417]","(-0.105, 0.4]","(0.0, 0.61]","(-9.9, 14.333]","(0.015, 0.2]","(0.01, 0.09]","(-0.223, 0.0]","(-0.05, 0.0833]","(-0.075, 0.903]","(-0.025, 0.277]","(-0.0765, 0.23]","(-0.0165, 0.0573]","(0.0146, 0.0897]","(0.0, 0.607]",0,"(0.0496, 0.665]"
1,-1,"(-0.5, 6.0]","(-0.333, 0.0]","(0.417, 0.833]","(0.4, 0.905]","(0.61, 1.22]","(14.333, 38.567]","(0.2, 0.385]","(0.09, 0.17]","(-0.447, -0.223]","(-0.183, -0.05]","(0.903, 1.882]","(0.277, 0.578]","(0.23, 0.537]","(0.0573, 0.131]","(0.0146, 0.0897]","(0.607, 1.214]",-1,"(0.665, 1.281]"
2,1,"(-7.0, -0.5]","(-0.333, 0.0]","(-0.417, 0.0]","(-0.61, -0.105]","(-0.61, 0.0]","(-34.133, -9.9]","(-0.355, -0.17]","(-0.15, -0.07]","(0.0, 0.223]","(-0.05, 0.0833]","(-1.053, -0.075]","(-0.327, -0.025]","(-0.383, -0.0765]","(-0.0903, -0.0165]","(-0.0605, 0.0146]","(-0.607, 0.0]",0,"(-1.181, -0.566]"
3,1,"(-7.0, -0.5]","(-0.333, 0.0]","(0.0, 0.417]","(-0.105, 0.4]","(0.0, 0.61]","(-9.9, 14.333]","(0.015, 0.2]","(0.01, 0.09]","(-0.223, 0.0]","(-0.183, -0.05]","(-0.075, 0.903]","(-0.025, 0.277]","(0.23, 0.537]","(-0.0165, 0.0573]","(0.0146, 0.0897]","(0.0, 0.607]",-1,"(0.665, 1.281]"
4,1,"(-20.039, -13.5]","(-0.333, 0.0]","(0.0, 0.417]","(-0.105, 0.4]","(0.0, 0.61]","(-9.9, 14.333]","(0.015, 0.2]","(0.09, 0.17]","(-0.223, 0.0]","(-0.05, 0.0833]","(-0.075, 0.903]","(-0.025, 0.277]","(-0.0765, 0.23]","(-0.0903, -0.0165]","(0.0146, 0.0897]","(0.0, 0.607]",1,"(-0.566, 0.0496]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3782,-1,"(-0.5, 6.0]","(-0.333, 0.0]","(-0.417, 0.0]","(-0.61, -0.105]","(-0.61, 0.0]","(-34.133, -9.9]","(-0.355, -0.17]","(-0.07, 0.01]","(-0.223, 0.0]","(-0.317, -0.183]","(-0.075, 0.903]","(-0.025, 0.277]","(-0.383, -0.0765]","(-0.0165, 0.0573]","(-0.136, -0.0605]","(-0.607, 0.0]",0,"(-0.566, 0.0496]"
3783,-1,"(-7.0, -0.5]","(-0.333, 0.0]","(0.0, 0.417]","(-0.105, 0.4]","(0.0, 0.61]","(-9.9, 14.333]","(0.015, 0.2]","(0.01, 0.09]","(-0.223, 0.0]","(-0.05, 0.0833]","(0.903, 1.882]","(-0.025, 0.277]","(-0.383, -0.0765]","(-0.0165, 0.0573]","(-0.0605, 0.0146]","(0.0, 0.607]",0,"(-1.181, -0.566]"
3784,-1,"(-0.5, 6.0]","(0.667, 1.0]","(-0.417, 0.0]","(-0.105, 0.4]","(-0.61, 0.0]","(-9.9, 14.333]","(-0.17, 0.015]","(0.01, 0.09]","(-0.223, 0.0]","(-0.05, 0.0833]","(-0.075, 0.903]","(-0.025, 0.277]","(-0.0765, 0.23]","(-0.0165, 0.0573]","(-0.0605, 0.0146]","(-0.607, 0.0]",0,"(-0.566, 0.0496]"
3785,-1,"(6.0, 12.5]","(0.0, 0.333]","(-0.833, -0.417]","(-0.61, -0.105]","(-1.22, -0.61]","(-34.133, -9.9]","(-0.355, -0.17]","(-0.07, 0.01]","(0.0, 0.223]","(-0.05, 0.0833]","(-1.053, -0.075]","(-0.327, -0.025]","(-0.383, -0.0765]","(-0.0165, 0.0573]","(-0.136, -0.0605]","(-1.214, -0.607]",1,"(-0.566, 0.0496]"


In [17]:
X = cs_binned.drop(["winner"],axis=1)
y = cs_binned["winner"]
np.random.seed(1)
Xtrain,ytrain,Xtest,ytest=helper.train_test_split(X,y)
testImportances = helper.exercise_7(Xtrain,ytrain,Xtest,ytest)
testImportances

{'world_rank': 0.003827751196172291,
 'h2h_win_perc': 0.001222753854332903,
 'rating': -0.0011164274322168488,
 'impact': -0.0024986709197235557,
 'kdr': -0.0037214247740563478,
 'dmr': 0.0,
 'kpr': -0.0032429558745348253,
 'apr': -0.0010101010101009056,
 'dpr': -0.000956937799043045,
 'spr': -0.001222753854332792,
 'opk_ratio': -0.0024986709197235557,
 'opk_rating': -0.0017012227538543145,
 'wins_perc_after_fk': -0.00419989367357787,
 'fk_perc_in_wins': -0.0022328548644338087,
 'multikill_perc': 0.0003721424774056903,
 'rating_at_least_one_perc': 0.00340244550770874,
 'is_sniper': 0.00010632642211594323,
 'clutch_win_perc': 5.3163211057971616e-05}

Our test importances found that the most significant variables were win percentages after first kill, world rank, and kill-death ratio

#### Comparing to Imported Package

First we must format the data such that we only have integers and there are no negative values. To do this, I will make every interval become the left side of the interval, and then subtract the minimum from each number, so that the minimum becomes zero and all other values become positive.

In [83]:
Xcopy = X.copy()
for col in Xcopy.columns:
    if type(Xcopy.loc[0][col]) == type(pd.Interval(0, 1)):
        Xcopy[col] = Xcopy[col].apply(lambda x: x.left)
    Xcopy[col] = Xcopy[col].apply(lambda x: x - Xcopy[col].min())
Xcopy

,world_rank,h2h_win_perc,rating,impact,kdr,dmr,kpr,apr,dpr,spr,opk_ratio,opk_rating,wins_perc_after_fk,fk_perc_in_wins,multikill_perc,rating_at_least_one_perc,is_sniper,clutch_win_perc
0,6.539,0.669,1.253,1.518,1.834,72.845,0.556,0.24,0.448,0.4010,2.941,0.907,0.9225,0.2215,0.2256,1.825,2,1.8506
1,19.539,0.669,1.670,2.023,2.444,97.078,0.741,0.32,0.224,0.2680,3.919,1.209,1.2290,0.2953,0.2256,2.432,1,2.4660
2,13.039,0.669,0.836,1.013,1.224,48.612,0.186,0.08,0.671,0.4010,1.963,0.605,0.6160,0.1477,0.1505,1.218,2,0.6200
3,13.039,0.669,1.253,1.518,1.834,72.845,0.556,0.24,0.448,0.2680,2.941,0.907,1.2290,0.2215,0.2256,1.825,1,2.4660
4,0.000,0.669,1.253,1.518,1.834,72.845,0.556,0.32,0.448,0.4010,2.941,0.907,0.9225,0.1477,0.2256,1.825,3,1.2350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3782,19.539,0.669,0.836,1.013,1.224,48.612,0.186,0.16,0.448,0.1340,2.941,0.907,0.6160,0.2215,0.0750,1.218,2,1.2350
3783,13.039,0.669,1.253,1.518,1.834,72.845,0.556,0.24,0.448,0.4010,3.919,0.907,0.6160,0.2215,0.1505,1.825,2,0.6200
3784,19.539,1.669,0.836,1.518,1.224,72.845,0.371,0.24,0.448,0.4010,2.941,0.907,0.9225,0.2215,0.1505,1.218,2,1.2350
3785,26.039,1.002,0.420,1.013,0.614,48.612,0.186,0.16,0.671,0.4010,1.963,0.605,0.6160,0.2215,0.0750,0.611,3,1.2350


Now we can test our data to see the imported package's accuracy

In [84]:
from sklearn.naive_bayes import CategoricalNB
clf = CategoricalNB()
Xtrain,ytrain,Xtest,ytest=helper.train_test_split(Xcopy,y)
clf.fit(Xtrain, ytrain)
clf.score(Xtest, ytest)

Not bad! We're only off by ~1% accuracy, a margin I'm more than willing to settle for.

In [10]:
prob = helper.specific_class_conditional(titanic_df['Sex'],'female',titanic_df['Survived'],0)
prob

0.14754098360655737

In [19]:
X = cs_binned.drop(["winner"],axis=1)
y = cs_binned["winner"]
probs = helper.class_conditional(X,y)
probs
X.head()

,world_rank,h2h_win_perc,rating,impact,kdr,dmr,kpr,apr,dpr,spr,opk_ratio,opk_rating,wins_perc_after_fk,fk_perc_in_wins,multikill_perc,rating_at_least_one_perc,is_sniper,clutch_win_perc
0,"(-13.5, -7.0]","(-0.333, 0.0]","(0.0, 0.417]","(-0.105, 0.4]","(0.0, 0.61]","(-9.9, 14.333]","(0.015, 0.2]","(0.01, 0.09]","(-0.223, 0.0]","(-0.05, 0.0833]","(-0.075, 0.903]","(-0.025, 0.277]","(-0.0765, 0.23]","(-0.0165, 0.0573]","(0.0146, 0.0897]","(0.0, 0.607]",0,"(0.0496, 0.665]"
1,"(-0.5, 6.0]","(-0.333, 0.0]","(0.417, 0.833]","(0.4, 0.905]","(0.61, 1.22]","(14.333, 38.567]","(0.2, 0.385]","(0.09, 0.17]","(-0.447, -0.223]","(-0.183, -0.05]","(0.903, 1.882]","(0.277, 0.578]","(0.23, 0.537]","(0.0573, 0.131]","(0.0146, 0.0897]","(0.607, 1.214]",-1,"(0.665, 1.281]"
2,"(-7.0, -0.5]","(-0.333, 0.0]","(-0.417, 0.0]","(-0.61, -0.105]","(-0.61, 0.0]","(-34.133, -9.9]","(-0.355, -0.17]","(-0.15, -0.07]","(0.0, 0.223]","(-0.05, 0.0833]","(-1.053, -0.075]","(-0.327, -0.025]","(-0.383, -0.0765]","(-0.0903, -0.0165]","(-0.0605, 0.0146]","(-0.607, 0.0]",0,"(-1.181, -0.566]"
3,"(-7.0, -0.5]","(-0.333, 0.0]","(0.0, 0.417]","(-0.105, 0.4]","(0.0, 0.61]","(-9.9, 14.333]","(0.015, 0.2]","(0.01, 0.09]","(-0.223, 0.0]","(-0.183, -0.05]","(-0.075, 0.903]","(-0.025, 0.277]","(0.23, 0.537]","(-0.0165, 0.0573]","(0.0146, 0.0897]","(0.0, 0.607]",-1,"(0.665, 1.281]"
4,"(-20.039, -13.5]","(-0.333, 0.0]","(0.0, 0.417]","(-0.105, 0.4]","(0.0, 0.61]","(-9.9, 14.333]","(0.015, 0.2]","(0.09, 0.17]","(-0.223, 0.0]","(-0.05, 0.0833]","(-0.075, 0.903]","(-0.025, 0.277]","(-0.0765, 0.23]","(-0.0903, -0.0165]","(0.0146, 0.0897]","(0.0, 0.607]",1,"(-0.566, 0.0496]"


In [18]:
X = cs_binned_d.drop(["winner"],axis=1)
y = cs_binned_d["winner"]
X.head()

NameError: name 'cs_binned_d' is not defined

In [20]:
probs = helper.class_conditional(X,y)
priors = helper.compute_priors(y)
x = cs_binned_d.drop("winner",axis=1).loc[2]
x

world_rank                       (-0.5, 6.0]
h2h_win_perc                   (-0.333, 0.0]
rating                        (0.417, 0.833]
impact                          (0.4, 0.905]
kdr                             (0.61, 1.22]
dmr                         (14.333, 38.567]
kpr                             (0.2, 0.385]
apr                             (0.09, 0.17]
dpr                         (-0.447, -0.223]
spr                          (-0.183, -0.05]
opk_ratio                     (0.903, 1.882]
opk_rating                    (0.277, 0.578]
wins_perc_after_fk             (0.23, 0.537]
fk_perc_in_wins              (0.0653, 0.141]
multikill_perc              (0.0146, 0.0897]
rating_at_least_one_perc      (0.607, 1.214]
is_sniper                               -1.0
clutch_win_perc               (0.665, 1.281]
Name: 2, dtype: object

In [21]:
dummyNode

world_rank                       (-0.5, 6.0]
h2h_win_perc                   (-0.333, 0.0]
rating                        (0.417, 0.833]
impact                         (0.905, 1.41]
kdr                             (0.61, 1.22]
dmr                           (38.567, 62.8]
kpr                            (0.385, 0.57]
apr                             (0.01, 0.09]
dpr                            (-0.223, 0.0]
spr                          (-0.05, 0.0833]
opk_ratio                      (1.882, 2.86]
opk_rating                     (0.578, 0.88]
wins_perc_after_fk           (-0.0765, 0.23]
fk_perc_in_wins               (0.141, 0.217]
multikill_perc              (0.0146, 0.0897]
rating_at_least_one_perc      (0.607, 1.214]
is_sniper                                1.0
clutch_win_perc             (-0.566, 0.0496]
Name: 0, dtype: object

In [22]:
post_probs = helper.posteriors(probs,priors,dummyNode)
post_probs

{'winner=-1.0|world_rank=(-0.5, 6.0],h2h_win_perc=(-0.333, 0.0],rating=(0.417, 0.833],impact=(0.905, 1.41],kdr=(0.61, 1.22],dmr=(38.567, 62.8],kpr=(0.385, 0.57],apr=(0.01, 0.09],dpr=(-0.223, 0.0],spr=(-0.05, 0.0833],opk_ratio=(1.882, 2.86],opk_rating=(0.578, 0.88],wins_perc_after_fk=(-0.0765, 0.23],fk_perc_in_wins=(0.141, 0.217],multikill_perc=(0.0146, 0.0897],rating_at_least_one_perc=(0.607, 1.214],is_sniper=1.0,clutch_win_perc=(-0.566, 0.0496]': 2.674851255317384e-05,
 'winner=1.0|world_rank=(-0.5, 6.0],h2h_win_perc=(-0.333, 0.0],rating=(0.417, 0.833],impact=(0.905, 1.41],kdr=(0.61, 1.22],dmr=(38.567, 62.8],kpr=(0.385, 0.57],apr=(0.01, 0.09],dpr=(-0.223, 0.0],spr=(-0.05, 0.0833],opk_ratio=(1.882, 2.86],opk_rating=(0.578, 0.88],wins_perc_after_fk=(-0.0765, 0.23],fk_perc_in_wins=(0.141, 0.217],multikill_perc=(0.0146, 0.0897],rating_at_least_one_perc=(0.607, 1.214],is_sniper=1.0,clutch_win_perc=(-0.566, 0.0496]': 0.9999732514874468}

Here is one more test you should check. Your code should return 50/50 if given a value that is not in the empirical distribution.

In [16]:
x = titanic_df.drop("Survived",axis=1).loc[2]
x = x.copy()
x['Age'] = 200
x

Pclass         3
Sex       female
Age          200
Name: 2, dtype: object

In [17]:
post_probs = helper.posteriors(probs,priors,x)
post_probs

{'Survived=0|Pclass=3,Sex=female,Age=200': 0.5,
 'Survived=1|Pclass=3,Sex=female,Age=200': 0.5}

In [21]:
import numpy as np
np.random.seed(2)
Xtrain,ytrain,Xtest,ytest=helper.train_test_split(X,y)
print('Xtrain')
display(Xtrain.head())
print('ytrain')
display(ytrain.head())
print('Xtest')
display(Xtest.head())
print('ytest')
display(ytest.head())

Xtrain


,world_rank,h2h_win_perc,rating,impact,kdr,dmr,kpr,apr,dpr,spr,opk_ratio,opk_rating,wins_perc_after_fk,fk_perc_in_wins,multikill_perc,rating_at_least_one_perc,is_sniper,clutch_win_perc
3401,"(-0.5, 6.0]","(-0.333, 0.0]","(0.0, 0.417]","(-0.105, 0.4]","(0.0, 0.61]","(-9.9, 14.333]","(-0.17, 0.015]","(0.01, 0.09]","(-0.223, 0.0]","(-0.05, 0.0833]","(-0.075, 0.903]","(-0.025, 0.277]","(-0.0765, 0.23]","(-0.0165, 0.0573]","(-0.0605, 0.0146]","(0.0, 0.607]",0,"(0.0496, 0.665]"
3343,"(-0.5, 6.0]","(0.0, 0.333]","(-0.417, 0.0]","(-0.105, 0.4]","(-0.61, 0.0]","(-9.9, 14.333]","(-0.17, 0.015]","(-0.15, -0.07]","(0.0, 0.223]","(-0.05, 0.0833]","(-1.053, -0.075]","(-0.025, 0.277]","(-0.383, -0.0765]","(0.0573, 0.131]","(-0.0605, 0.0146]","(-0.607, 0.0]",-1,"(0.0496, 0.665]"
1862,"(-0.5, 6.0]","(0.0, 0.333]","(-0.417, 0.0]","(-0.61, -0.105]","(-0.61, 0.0]","(-9.9, 14.333]","(-0.17, 0.015]","(-0.07, 0.01]","(0.223, 0.447]","(-0.05, 0.0833]","(-1.053, -0.075]","(-0.025, 0.277]","(-0.383, -0.0765]","(0.0573, 0.131]","(-0.0605, 0.0146]","(-0.607, 0.0]",0,"(-1.181, -0.566]"
1770,"(-20.039, -13.5]","(-0.333, 0.0]","(0.0, 0.417]","(-0.105, 0.4]","(0.0, 0.61]","(-9.9, 14.333]","(0.015, 0.2]","(0.01, 0.09]","(-0.223, 0.0]","(-0.183, -0.05]","(-1.053, -0.075]","(-0.025, 0.277]","(0.23, 0.537]","(-0.0903, -0.0165]","(0.0146, 0.0897]","(0.0, 0.607]",1,"(-0.566, 0.0496]"
1010,"(-7.0, -0.5]","(0.0, 0.333]","(0.0, 0.417]","(-0.105, 0.4]","(0.61, 1.22]","(14.333, 38.567]","(0.2, 0.385]","(-0.07, 0.01]","(-0.223, 0.0]","(-0.05, 0.0833]","(-0.075, 0.903]","(-0.025, 0.277]","(-0.0765, 0.23]","(-0.0165, 0.0573]","(0.0146, 0.0897]","(0.0, 0.607]",0,"(-0.566, 0.0496]"


ytrain


3401    1
3343    1
1862   -1
1770   -1
1010   -1
Name: winner, dtype: int64

Xtest


,world_rank,h2h_win_perc,rating,impact,kdr,dmr,kpr,apr,dpr,spr,opk_ratio,opk_rating,wins_perc_after_fk,fk_perc_in_wins,multikill_perc,rating_at_least_one_perc,is_sniper,clutch_win_perc
2516,"(-13.5, -7.0]","(0.333, 0.667]","(0.0, 0.417]","(0.4, 0.905]","(0.0, 0.61]","(14.333, 38.567]","(0.015, 0.2]","(0.09, 0.17]","(-0.223, 0.0]","(-0.05, 0.0833]","(-0.075, 0.903]","(-0.025, 0.277]","(-0.0765, 0.23]","(-0.0903, -0.0165]","(0.0146, 0.0897]","(0.0, 0.607]",-1,"(-0.566, 0.0496]"
3408,"(-7.0, -0.5]","(0.0, 0.333]","(0.0, 0.417]","(-0.105, 0.4]","(0.0, 0.61]","(-9.9, 14.333]","(-0.17, 0.015]","(0.01, 0.09]","(-0.223, 0.0]","(-0.183, -0.05]","(-0.075, 0.903]","(-0.025, 0.277]","(-0.0765, 0.23]","(-0.0165, 0.0573]","(-0.0605, 0.0146]","(0.0, 0.607]",-1,"(-0.566, 0.0496]"
1605,"(-13.5, -7.0]","(0.333, 0.667]","(0.417, 0.833]","(-0.105, 0.4]","(0.61, 1.22]","(14.333, 38.567]","(0.2, 0.385]","(-0.15, -0.07]","(-0.223, 0.0]","(-0.183, -0.05]","(0.903, 1.882]","(0.277, 0.578]","(-0.0765, 0.23]","(-0.0165, 0.0573]","(0.0897, 0.165]","(0.607, 1.214]",0,"(-1.181, -0.566]"
2843,"(-7.0, -0.5]","(0.0, 0.333]","(0.0, 0.417]","(-0.105, 0.4]","(0.0, 0.61]","(-9.9, 14.333]","(0.015, 0.2]","(0.09, 0.17]","(-0.223, 0.0]","(-0.05, 0.0833]","(-0.075, 0.903]","(-0.025, 0.277]","(-0.0765, 0.23]","(-0.0165, 0.0573]","(0.0146, 0.0897]","(0.0, 0.607]",0,"(0.0496, 0.665]"
1487,"(-7.0, -0.5]","(-0.333, 0.0]","(-0.417, 0.0]","(-0.105, 0.4]","(0.0, 0.61]","(-9.9, 14.333]","(0.015, 0.2]","(-0.15, -0.07]","(0.0, 0.223]","(-0.183, -0.05]","(-0.075, 0.903]","(-0.025, 0.277]","(-0.0765, 0.23]","(-0.0165, 0.0573]","(0.0146, 0.0897]","(0.0, 0.607]",0,"(-0.566, 0.0496]"


ytest


2516    1
3408   -1
1605    1
2843    1
1487    1
Name: winner, dtype: int64

#### Test Accuracies
Testing accuracy for various bin sizes in order to find which bin size is ideal.

In [21]:
Xtrain.drop_duplicates().sort_values('Age')
ytest

716    1
154    0
843    0
24     0
294    0
      ..
534    0
584    0
493    0
527    0
168    0
Name: Survived, Length: 445, dtype: int64

In [20]:
# 0.7820224719101123
np.random.seed(0)
Xtrain,ytrain,Xtest,ytest=helper.train_test_split(X,y)
x = Xtest.iloc[0]
x

world_rank                       (-13.5, -7.0]
h2h_win_perc                      (0.667, 1.0]
rating                            (0.0, 0.417]
impact                            (0.4, 0.905]
kdr                                (0.0, 0.61]
dmr                             (-9.9, 14.333]
kpr                               (0.015, 0.2]
apr                              (-0.07, 0.01]
dpr                              (-0.223, 0.0]
spr                            (-0.05, 0.0833]
opk_ratio                      (-0.075, 0.903]
opk_rating                     (-0.025, 0.277]
wins_perc_after_fk             (-0.0765, 0.23]
fk_perc_in_wins             (-0.0903, -0.0165]
multikill_perc                (0.0146, 0.0897]
rating_at_least_one_perc          (0.0, 0.607]
is_sniper                                    1
clutch_win_perc               (-0.566, 0.0496]
Name: 897, dtype: object

In [23]:
bin_cols = cs_norm.columns
bin_cols = bin_cols.drop(['winner', 'is_sniper'])
accs = {}
ntrials = 10
for i in range(5, 20):
    accs[i] = 0
    cs_binned = helper.bin_cols(cs_norm, bin_cols, bins = i)
    for n in range(ntrials):
        Xtrain,ytrain,Xtest,ytest=helper.train_test_split(cs_binned.drop('winner', axis=1), cs_binned.winner)
        accs[i] += (helper.exercise_6(Xtrain,ytrain,Xtest,ytest))
    accs[i] = accs[i]/ntrials
accs

{5: 0.5869750132908027,
 6: 0.5900584795321637,
 7: 0.5898989898989899,
 8: 0.5887825624667731,
 9: 0.5850079744816586,
 10: 0.5874003189792664,
 11: 0.5823498139287613,
 12: 0.5861775651249336,
 13: 0.5871345029239765,
 14: 0.5901116427432217,
 15: 0.5790536948431685,
 16: 0.5843168527379053,
 17: 0.5890483785220627,
 18: 0.5812865497076023,
 19: 0.5860180754917597}

In [18]:
#The function can take a long time to run so I've saved the results into this box
if False:
    {5: 0.5869750132908027,
     6: 0.5900584795321637,
     7: 0.5898989898989899,
     8: 0.5887825624667731,
     9: 0.5850079744816586,
     10: 0.5874003189792664,
     11: 0.5823498139287613,
     12: 0.5861775651249336,
     13: 0.5871345029239765,
     14: 0.5901116427432217,
     15: 0.5790536948431685,
     16: 0.5843168527379053,
     17: 0.5890483785220627,
     18: 0.5812865497076023,
     19: 0.5860180754917597}

In [21]:
accuracy = helper.exercise_6(Xtrain,ytrain,Xtest,ytest)
accuracy

0.5943646996278575

A single accuracy run for our current bin size (6) and data set

In [ ]:
accuracy = sum(ytest == 1)/ytest.size
accuracy

0.518341307814992

Is that better than guessing that team 1 won each match? What is the accuracy on the test set if you guessed team 1 won every match?

Compared to predicting that team 1 won every match, dynamically predicting based on the Bayesian classifier gave us ~7% higher accuracy


#### Excercise 7. Create a function that returns the test based feature importance for our Bayesian classifier.

In [12]:
bin_cols = cs_norm.columns
bin_cols = bin_cols.drop(['winner', 'is_sniper'])
cs_binned = helper.bin_cols(cs_norm, bin_cols, bins = 6)
X = cs_binned.drop('winner', axis = 1)
y = cs_binned["winner"]

In [13]:
np.random.seed(0)
Xtrain,ytrain,Xtest,ytest=helper.train_test_split(X,y)
importances = helper.exercise_7(Xtrain,ytrain,Xtest,ytest)
importances

{'world_rank': 0.008240297713982048,
 'h2h_win_perc': 0.004678362573099504,
 'rating': -0.002020202020202033,
 'impact': 0.0037214247740564588,
 'kdr': -0.00010632642211594323,
 'dmr': 0.0011164274322169598,
 'kpr': 0.002711323763955442,
 'apr': 0.0018607123870282294,
 'dpr': 0.0013290802764487353,
 'spr': 0.0009037745879851844,
 'opk_ratio': 0.0006379585326954373,
 'opk_rating': 0.00047846889952141147,
 'wins_perc_after_fk': 0.00031897926634771867,
 'fk_perc_in_wins': 0.0017543859649122862,
 'multikill_perc': 0.0010101010101010166,
 'rating_at_least_one_perc': 0.0015948963317384823,
 'is_sniper': 0.0009037745879851844,
 'clutch_win_perc': 0.0011164274322169598}

In [19]:
#This can also take a long time to run so the results have been saved into this box

importances = {'world_rank': 0.008240297713982048,
 'h2h_win_perc': 0.004678362573099504,
 'rating': -0.002020202020202033,
 'impact': 0.0037214247740564588,
 'kdr': -0.00010632642211594323,
 'dmr': 0.0011164274322169598,
 'kpr': 0.002711323763955442,
 'apr': 0.0018607123870282294,
 'dpr': 0.0013290802764487353,
 'spr': 0.0009037745879851844,
 'opk_ratio': 0.0006379585326954373,
 'opk_rating': 0.00047846889952141147,
 'wins_perc_after_fk': 0.00031897926634771867,
 'fk_perc_in_wins': 0.0017543859649122862,
 'multikill_perc': 0.0010101010101010166,
 'rating_at_least_one_perc': 0.0015948963317384823,
 'is_sniper': 0.0009037745879851844,
 'clutch_win_perc': 0.0011164274322169598}
importances.sort_values(ascending = False)

AttributeError: 'dict' object has no attribute 'sort_values'

#### Excercise 8. Create a function that returns the train based feature importance for our Bayesian classifier.

In [14]:
np.random.seed(0)
Xtrain,ytrain,Xtest,ytest=helper.train_test_split(X,y)
importances = helper.exercise_8(Xtrain,ytrain,Xtest,ytest)
importances

{'world_rank': 0.00839978734715563,
 'h2h_win_perc': 0.0057947900053161305,
 'rating': -0.002099946836788824,
 'impact': 0.0017809675704412165,
 'kdr': -0.0016214779372670796,
 'dmr': 0.002631578947368318,
 'kpr': 0.0027379053694842614,
 'apr': 0.002339181286549641,
 'dpr': -2.6581605528819274e-05,
 'spr': 0.001036682615629947,
 'opk_ratio': -0.00039872408293439854,
 'opk_rating': 0.00021265284423199748,
 'wins_perc_after_fk': 0.0009037745879852954,
 'fk_perc_in_wins': 0.0017809675704414385,
 'multikill_perc': 0.0007442849548114916,
 'rating_at_least_one_perc': 0.0012493354598619444,
 'is_sniper': 0.0017543859649122862,
 'clutch_win_perc': 0.0018607123870282294}

In [44]:
#This can also take a long time to run so the results have been saved into this box

{'world_rank': 0.00839978734715563,
 'h2h_win_perc': 0.0057947900053161305,
 'rating': -0.002099946836788824,
 'impact': 0.0017809675704412165,
 'kdr': -0.0016214779372670796,
 'dmr': 0.002631578947368318,
 'kpr': 0.0027379053694842614,
 'apr': 0.002339181286549641,
 'dpr': -2.6581605528819274e-05,
 'spr': 0.001036682615629947,
 'opk_ratio': -0.00039872408293439854,
 'opk_rating': 0.00021265284423199748,
 'wins_perc_after_fk': 0.0009037745879852954,
 'fk_perc_in_wins': 0.0017809675704414385,
 'multikill_perc': 0.0007442849548114916,
 'rating_at_least_one_perc': 0.0012493354598619444,
 'is_sniper': 0.0017543859649122862,
 'clutch_win_perc': 0.0018607123870282294}
 

{'world_rank': 0.00839978734715563,
 'h2h_win_perc': 0.0057947900053161305,
 'rating': -0.002099946836788824,
 'impact': 0.0017809675704412165,
 'kdr': -0.0016214779372670796,
 'dmr': 0.002631578947368318,
 'kpr': 0.0027379053694842614,
 'apr': 0.002339181286549641,
 'dpr': -2.6581605528819274e-05,
 'spr': 0.001036682615629947,
 'opk_ratio': -0.00039872408293439854,
 'opk_rating': 0.00021265284423199748,
 'wins_perc_after_fk': 0.0009037745879852954,
 'fk_perc_in_wins': 0.0017809675704414385,
 'multikill_perc': 0.0007442849548114916,
 'rating_at_least_one_perc': 0.0012493354598619444,
 'is_sniper': 0.0017543859649122862,
 'clutch_win_perc': 0.0018607123870282294}